In [1]:
#data preprocessing
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

np.random.seed(42)

In [ ]:
data_file=os.path.join(os.getcwd(),"data/shuffled-full-set-hashed.csv")
dataframe_all = pd.read_csv(data_file, sep=",")
dataframe_all.columns = ["document_label", "word_values"]
dataframe_all=dataframe_all.dropna(subset = ['word_values'])


In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(dataframe_all['word_values'], dataframe_all['document_label']):
    strat_train_set = dataframe_all.loc[train_index]
    strat_test_set = dataframe_all.loc[test_index]

/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """
/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [4]:
strat_train_set=strat_train_set.dropna(subset = ['word_values'])
strat_test_set=strat_test_set.dropna(subset = ['word_values'])

In [5]:
x_train, y_train=strat_train_set['word_values'], strat_train_set['document_label']
x_test, y_test=strat_test_set['word_values'], strat_test_set['document_label']

In [6]:
y_train[0]

'RETURNED CHECK'

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
label_encoder = LabelEncoder()
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

class LabelEncoderPipelineFriendly(LabelEncoder):
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelEncoderPipelineFriendly, self).fit(X)
    def transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).fit(X).transform(X)

class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return np.transpose(np.matrix(data))


num_pipeline = Pipeline([
        ('selector', DataFrameSelector('word_values')),
        ('vect', TfidfVectorizer(max_df=0.95, min_df=2, max_features=1037929)),
        ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector('document_label')),
        ('encoder', LabelEncoderPipelineFriendly()),
    ('caster', ArrayCaster()),
    ]) 

from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
          #("cat_pipeline", cat_pipeline),
    ])


In [15]:
x_train, y_train=full_pipeline.fit_transform(strat_train_set), label_encoder.fit_transform(strat_train_set['document_label'])
x_test, y_test=full_pipeline.transform(strat_test_set), label_encoder.fit_transform(strat_test_set['document_label'])

In [9]:
y_train[0]

12

In [6]:
from sklearn.externals import joblib
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC

#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    
    return f1_score(target, y_pred,labels=range(14),average='micro'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    

    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print f1, acc
    print "F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print "F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc)

In [13]:
clf_A = LogisticRegression(C=14, solver='lbfgs',multi_class='multinomial',random_state = 42)
train_predict(clf_A, x_train, y_train, x_test, y_test)
print ''
joblib.dump(clf_A,"class_lr.pkl")

Trained model in 110.9364 seconds
Made predictions in 0.2914 seconds.
0.962632369777 0.962632369777
F1 score and accuracy score for training set: 0.9626 , 0.9626.
Made predictions in 0.0328 seconds.
F1 score and accuracy score for test set: 0.8879 , 0.8879.



['class_lr.pkl']

In [ ]:
# Initialize the three models 
print "Start"
# clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)
joblib.dump(clf_C,"class_xg.pkl")

# train_predict(clf_B, x_train, y_train, x_test, y_test)
# print ''
train_predict(clf_C, x_train, y_train, x_test, y_test)
print ''

Start


In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,labels=range(14),average='micro')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(x_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, x_train, y_train)
print "F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc)
    
f1, acc = predict_labels(clf, x_test, y_test)
print "F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc)

/root/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/root/miniconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


KeyboardInterrupt: 

In [38]:

#clf_B = SVC(random_state = 912,C=10,gamma=0.001, kernel='rbf')
print "Loading"
clf_B = joblib.load('classi_rf.pkl') 
print "Loaded"
# Print the results of prediction for both training and testing
f1, acc = predict_labels(clf_B, x_train, y_train)
print f1, acc
print "F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc)

f1, acc = predict_labels(clf_B, x_test, y_test)
print "F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc)


Loading


ValueError: non-string names in Numpy dtype unpickling